In [1]:
import torch
import jax

In [2]:
a = torch.load("/Users/rasmus/data/pretrained_models/my_models/last_l1.ckpt", map_location=torch.device("cpu"))
b = torch.load("/Users/rasmus/data/pretrained_models/HAGAN/G_iter80000.pth")

In [3]:
a["state_dict"].keys()

odict_keys(['G.fc1.weight', 'G.fc1.bias', 'G.tp_conv1.weight', 'G.tp_conv1.bias', 'G.bn1.weight', 'G.bn1.bias', 'G.tp_conv2.weight', 'G.tp_conv2.bias', 'G.bn2.weight', 'G.bn2.bias', 'G.tp_conv3.weight', 'G.tp_conv3.bias', 'G.bn3.weight', 'G.bn3.bias', 'G.tp_conv4.weight', 'G.tp_conv4.bias', 'G.bn4.weight', 'G.bn4.bias', 'G.tp_conv5.weight', 'G.tp_conv5.bias', 'G.bn5.weight', 'G.bn5.bias', 'G.tp_conv6.weight', 'G.tp_conv6.bias', 'G.bn6.weight', 'G.bn6.bias', 'G.tp_conv7.weight', 'G.tp_conv7.bias', 'G.sub_G.tp_conv1.weight', 'G.sub_G.tp_conv1.bias', 'G.sub_G.bn1.weight', 'G.sub_G.bn1.bias', 'G.sub_G.tp_conv2.weight', 'G.sub_G.tp_conv2.bias', 'G.sub_G.bn2.weight', 'G.sub_G.bn2.bias', 'G.sub_G.tp_conv3.weight', 'G.sub_G.tp_conv3.bias', 'D.conv1.weight', 'D.conv1.bias', 'D.conv1.u0', 'D.conv1.sv0', 'D.conv2.weight', 'D.conv2.bias', 'D.conv2.u0', 'D.conv2.sv0', 'D.conv3.weight', 'D.conv3.bias', 'D.conv3.u0', 'D.conv3.sv0', 'D.conv4.weight', 'D.conv4.bias', 'D.conv4.u0', 'D.conv4.sv0', 'D.con

In [4]:
def tree_key_map(f, tree):
    if isinstance(tree, dict):
        return {f(k): tree_key_map(f, v) for k, v in tree.items()}
    elif isinstance(tree, list):
        return [tree_key_map(f, v) for v in tree]
    else:
        return tree

tree_map = jax.tree.map

In [5]:
# tree_map(lambda x: x.shape, b)
b_fixed = tree_key_map(lambda x: "G." + x.replace("module.", ""), b["model"])

In [6]:
# dict(tree_map(lambda x: x.size(), a))

In [7]:
for k, v in b_fixed.items():
    if k in a["state_dict"]:
        a["state_dict"][k] = v
    else:
        raise ValueError(f"Key {k} not found in a")

In [8]:
torch.save(a, "/Users/rasmus/data/pretrained_models/my_models/HAGAN_from_authors.ckpt")

## Test it out